# PySpark labo met oplossingen

(findspark niet meer nodig in de jupyterhub van nathan)

In [1]:
import findspark
findspark.init('/home/marie/spark')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

## tekst?

In [4]:
data = spark.read.csv('kc_house_data.csv',inferSchema=True,header=True)

## eventueel zelf kijken om er van een paar te veranderen naar strings dat de studentjes naar het juiste moeten omzetten.

In [5]:
data.printSchema() #optioneel

root
 |-- id: long (nullable = true)
 |-- date: string (nullable = true)
 |-- price: decimal(7,0) (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: double (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: double (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)
 |-- condition: integer (nullable = true)
 |-- grade: integer (nullable = true)
 |-- sqft_above: integer (nullable = true)
 |-- sqft_basement: integer (nullable = true)
 |-- yr_built: integer (nullable = true)
 |-- yr_renovated: integer (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- sqft_living15: integer (nullable = true)
 |-- sqft_lot15: integer (nullable = true)



In [6]:
data = data.drop('date')
data = data.drop('long')
data = data.drop('lat')

In [7]:
data.show()

+----------+-------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------------+----------+
|        id|  price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|sqft_living15|sqft_lot15|
+----------+-------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------------+----------+
|7129300520| 221900|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|         1340|      5650|
|6414100192| 538000|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|    1951|        1991|  98125|         1690|      7639|
|5631500400| 180000|       2|      1.0|        770|   10000|   1.0|         0|   0|        3|    6| 

In [8]:
data.columns

['id',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'sqft_living15',
 'sqft_lot15']

### aangezien describe zo onoverzichtelijk is mss select er een paar en dan describe

In [9]:
data.describe().show()

+-------+--------------------+------------------+-----------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+
|summary|                  id|             price|         bedrooms|         bathrooms|       sqft_living|          sqft_lot|            floors|          waterfront|               view|         condition|             grade|        sqft_above|     sqft_basement|          yr_built|      yr_renovated|           zipcode|    sqft_living15|        sqft_lot15|
+-------+--------------------+------------------+-----------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-----------

# selecteren van kolommen
Je kan een kolom selecteren aan de hand van de methode:  
`df["column_name"]`  
selecteer de kolom condition

In [10]:
data['condition']

Column<condition>

Het verkregen type is een Column, maar met de functie `df.select('column_name')` krijg je een dataframe terug, die veelzijdiger is dan het type Column. Selecteer opnieuw de kolom condition maar nu als dataframe

In [11]:
data.select('condition')

DataFrame[condition: int]

Om de inhoud van de geselecteerde kolom weer te geven moet je de methode `.show` aanroepen.

In [12]:
data.select('condition').show()

+---------+
|condition|
+---------+
|        3|
|        3|
|        3|
|        5|
|        3|
|        3|
|        3|
|        3|
|        3|
|        3|
|        3|
|        4|
|        4|
|        4|
|        3|
|        3|
|        3|
|        4|
|        4|
|        4|
+---------+
only showing top 20 rows



`data.head(n)` geeft de eerste n rijen terug als lijst.  
print de eerste 5 rijen


In [13]:
data.head(5)

[Row(id=7129300520, price=Decimal('221900'), bedrooms=3, bathrooms=1.0, sqft_living=1180, sqft_lot=5650, floors=1.0, waterfront=0, view=0, condition=3, grade=7, sqft_above=1180, sqft_basement=0, yr_built=1955, yr_renovated=0, zipcode=98178, sqft_living15=1340, sqft_lot15=5650),
 Row(id=6414100192, price=Decimal('538000'), bedrooms=3, bathrooms=2.25, sqft_living=2570, sqft_lot=7242, floors=2.0, waterfront=0, view=0, condition=3, grade=7, sqft_above=2170, sqft_basement=400, yr_built=1951, yr_renovated=1991, zipcode=98125, sqft_living15=1690, sqft_lot15=7639),
 Row(id=5631500400, price=Decimal('180000'), bedrooms=2, bathrooms=1.0, sqft_living=770, sqft_lot=10000, floors=1.0, waterfront=0, view=0, condition=3, grade=6, sqft_above=770, sqft_basement=0, yr_built=1933, yr_renovated=0, zipcode=98028, sqft_living15=2720, sqft_lot15=8062),
 Row(id=2487200875, price=Decimal('604000'), bedrooms=4, bathrooms=3.0, sqft_living=1960, sqft_lot=5000, floors=1.0, waterfront=0, view=0, condition=5, grade=

je kan met `data.head()` ook een specifieke rij of een bepaalde waarde selecteren met behulp van:  
`data.head(n)[row_number][column_number]`  
*1) selecteer de 4de rij*  
*2) selecteer prijs op de 3de rij*

In [14]:
data.head(5)[3]

Row(id=2487200875, price=Decimal('604000'), bedrooms=4, bathrooms=3.0, sqft_living=1960, sqft_lot=5000, floors=1.0, waterfront=0, view=0, condition=5, grade=7, sqft_above=1050, sqft_basement=910, yr_built=1965, yr_renovated=0, zipcode=98136, sqft_living15=1360, sqft_lot15=5000)

In [15]:
data.head(5)[2][1]

Decimal('180000')

*Is dit een action of transformation?*

df.select met meerdere kolommen?
# vraag uitwerken

# Creating a new column and dropping columns

Je kan een nieuwe kolom aanmaken met de functie: `data.withColumn('new_column_name',data["column"])`  
data["column"] kan ook bewerkt worden bv: `data.withColumn('new_column_name',data["column"]/2)`  
een kolom kan ook verwijdert worden met `data.drop('column')`  
_verander nu alle sqft kolommen naar m2 (*0.0929) en verwijder de sqft kolommen._

In [17]:
def to_m2(new_name,old_column,dataset):
    dataset = dataset.withColumn(new_name,dataset[old_column]*0.0929)
    dataset = dataset.drop(old_column)
    return dataset

In [18]:
data = to_m2('m2_living','sqft_living',data)
data = to_m2('m2_lot','sqft_lot',data)
data = to_m2('m2_living15','sqft_living15',data)
data = to_m2('m2_lot15','sqft_lot15',data)

In [19]:
data.columns

['id',
 'price',
 'bedrooms',
 'bathrooms',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'm2_living',
 'm2_lot',
 'm2_living15',
 'm2_lot15']

# Filtering and grouping data

# MlLib gedeelte

Om machine learning te kunnen toepassen met MlLib verwacht spark een format dat 2 kolommen bevat met de namen: "label" en "features".  
De label kolom moet een numerische label bevatten, dit kan een numerische waarde zijn voor regressie of classificatie.  
De feature kolom moet een vector van alle features bevatten.  
In deze sectie zal je een dataframe omzetten naar het verwachte format en zal je de huisprijzen proberen te voorspellen met lineaire regressie.

In [20]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [21]:
data_features = data.drop('price')
data_features = data_features.drop('id')
columns = data_features.columns
print(columns)

['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'm2_living', 'm2_lot', 'm2_living15', 'm2_lot15']


In [22]:
assembler = VectorAssembler(inputCols=columns,outputCol='features')

In [23]:
output = assembler.transform(data)

In [24]:
output.head(1)

[Row(id=7129300520, price=Decimal('221900'), bedrooms=3, bathrooms=1.0, floors=1.0, waterfront=0, view=0, condition=3, grade=7, sqft_above=1180, sqft_basement=0, yr_built=1955, yr_renovated=0, zipcode=98178, m2_living=109.622, m2_lot=524.885, m2_living15=124.48599999999999, m2_lot15=524.885, features=DenseVector([3.0, 1.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1180.0, 0.0, 1955.0, 0.0, 98178.0, 109.622, 524.885, 124.486, 524.885]))]

In [25]:
final_data = output.select('features','price')
final_data.head()

Row(features=DenseVector([3.0, 1.0, 1.0, 0.0, 0.0, 3.0, 7.0, 1180.0, 0.0, 1955.0, 0.0, 98178.0, 109.622, 524.885, 124.486, 524.885]), price=Decimal('221900'))

In [26]:
train_data,test_data = final_data.randomSplit([0.8,0.2])

In [27]:
train_data.describe().show()

+-------+----------------+
|summary|           price|
+-------+----------------+
|  count|           17173|
|   mean|     539451.5882|
| stddev|367281.908283828|
|    min|           75000|
|    max|         7700000|
+-------+----------------+



In [28]:
test_data.describe().show()

+-------+----------------+
|summary|           price|
+-------+----------------+
|  count|            4440|
|   mean|     542550.1989|
| stddev|366559.097747102|
|    min|           80000|
|    max|         4489000|
+-------+----------------+



In [29]:
lr = LinearRegression(labelCol='price',regParam=0.1)
lr_model = lr.fit(train_data)
test_results = lr_model.evaluate(test_data)
test_results.residuals.show() #residuals is the difference between the predicted value and the label of the test data

+-------------------+
|          residuals|
+-------------------+
|  547474.4741438814|
|  75191.86543961428|
| 255868.23556314595|
|  454759.2136101872|
|-108691.93934419099|
| -94199.83247038815|
|-28779.600327363238|
|  135211.5608439762|
| 55329.473707691766|
| 29232.196342014708|
|  328623.7339273989|
|-42883.589010504074|
|  220960.6534954831|
|  43150.21118671913|
| 44321.796752921306|
|-127560.78113140352|
| -68000.33932255208|
|  155753.0955591146|
|-129895.33389155753|
|   229809.042029785|
+-------------------+
only showing top 20 rows



In [30]:
test_results.rootMeanSquaredError

216465.73240691365

In [31]:
test_results.r2

0.6511906405151969